In [1]:
# %% [markdown]
# # Normalização dos municípios e cruzamento com tabela IBGE (versão ajustada para estrutura: COD_UF, COD, NOME)

import pandas as pd
import glob
import os
import unidecode
from rapidfuzz import process

# %% 1. Carregar lista oficial do IBGE
url_ibge = "C:/Users/Valentine/Artigo_Mapitos/data_raw/data_auxiliar/IBGE_mun.csv"
ibge = pd.read_csv(url_ibge)

# Filtrar Tocantins (COD_UF = 17)
ibge_to = ibge[ibge["COD_UF"] == 17].copy()

# Normalizar nome
ibge_to["nome_norm"] = ibge_to["NOME"].str.lower().apply(unidecode.unidecode)

print("📘 Exemplo de municípios do TO:")
print(ibge_to.head())

# %% 2. Funções auxiliares
def normalizar_nome(nome):
    if pd.isna(nome):
        return ""
    return unidecode.unidecode(str(nome).strip().lower())

def melhor_match(municipio, lista_ref):
    if not municipio:
        return None
    match, score, _ = process.extractOne(municipio, lista_ref)
    return match if score > 80 else None

# %% 3. Processar CSVs
input_path = "C:/Users/Valentine/Artigo_Mapitos/data_raw/degradacao.csv"
arquivos = glob.glob(input_path)

print("📂 Arquivos encontrados:", arquivos)

dfs_processados = []

for arquivo in arquivos:
    df = pd.read_csv(arquivo)
    print(f"🔧 Processando {os.path.basename(arquivo)} ...")

    # Detectar a coluna de município (casos como 'municipio__', 'Município', etc)
    col_munic = next((c for c in df.columns if "munic" in c.lower()), None)

    if not col_munic:
        print("⚠️ Nenhuma coluna de município encontrada, pulando este arquivo.")
        continue

    # Normalizar nomes
    df["municipio_norm"] = df[col_munic].apply(normalizar_nome)

    # Merge direto com nomes normalizados
    merged = df.merge(ibge_to, left_on="municipio_norm", right_on="nome_norm", how="left")

    # Fuzzy matching para os que não bateram
    mask_na = merged["COD"].isna()
    if mask_na.any():
        lista_ref = ibge_to["nome_norm"].tolist()
        merged.loc[mask_na, "nome_match"] = merged.loc[mask_na, "municipio_norm"].apply(
            lambda x: melhor_match(x, lista_ref)
        )
        merged = merged.merge(
            ibge_to[["COD", "COD_UF", "NOME", "nome_norm"]],
            left_on="nome_match",
            right_on="nome_norm",
            how="left",
            suffixes=("", "_fuzzy"),
        )
        merged["codigo_final"] = merged["COD"].combine_first(merged["COD_fuzzy"])
        merged["uf_final"] = merged["COD_UF"].combine_first(merged["COD_UF_fuzzy"])
        merged["municipio_ibge"] = merged["NOME"].combine_first(merged["NOME_fuzzy"])
    else:
        merged["codigo_final"] = merged["COD"]
        merged["uf_final"] = merged["COD_UF"]
        merged["municipio_ibge"] = merged["NOME"]

    merged = merged.rename(columns={col_munic: "municipio_original"})
    dfs_processados.append(merged)

# %% 4. Consolidar e exportar
if dfs_processados:
    df_final = pd.concat(dfs_processados, ignore_index=True)
    print("✅ Registros consolidados:", len(df_final))
else:
    df_final = pd.DataFrame()
    print("⚠️ Nenhum arquivo processado.")

output_dir = "C:/Users/Valentine/Artigo_Mapitos/data_clean/01_Municipios_Normalizados"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "01_Deg_mun.csv")

df_final.to_csv(output_file, index=False, encoding="utf-8-sig")
print("📦 Arquivo final exportado para:", output_file)

# Agora cada linha tem:
# - municipio_original (do seu CSV)
# - municipio_norm (normalizado)
# - municipio_ibge (nome oficial IBGE)
# - codigo_final (código IBGE)
# - uf_final (UF)


📘 Exemplo de municípios do TO:
     COD_UF      COD                  NOME             nome_norm
311      17  1700251           Abreulândia           abreulandia
312      17  1700301         Aguiarnópolis         aguiarnopolis
313      17  1700350  Aliança do Tocantins  alianca do tocantins
314      17  1700400                 Almas                 almas
315      17  1700707              Alvorada              alvorada
📂 Arquivos encontrados: ['C:/Users/Valentine/Artigo_Mapitos/data_raw/degradacao.csv']
🔧 Processando degradacao.csv ...
✅ Registros consolidados: 1325
📦 Arquivo final exportado para: C:/Users/Valentine/Artigo_Mapitos/data_clean/01_Municipios_Normalizados\01_Deg_mun.csv
